In [8]:
import numpy as np
import pandas as pd
import glob
import random
import time
import keras

from keras.models import Sequential
from keras.layers import Dense

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from joblib import dump

In [19]:
## Importing data ##
records = pd.read_csv(r'C:\Users\User\Desktop\Stanley\time24\data\records24.txt', sep = ",", header = None)
labels = pd.read_csv(r'C:\Users\User\Desktop\Stanley\time24\data\labels24.txt', sep = ",", header = None)

print(records.notnull().values.all(), records.shape, labels.shape)

True (5974, 17) (5974, 1)


In [20]:
## Normalizing function ##
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [24]:
## Splitting training and validation sets ##
train, test, train_labels, test_labels = train_test_split(records, labels,
                                                          test_size=0.2, 
                                                          random_state=42)

In [25]:
## Process and reshaping data to fit the training models ##
norm_records = normalize(records)

header = ["Mean", "Stdev", "Over60", "O60U120", "Over120", "Skewness", "Kurtosis", "Entropy", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9"]
records.columns = header
norm_records.columns = header
labels.columns = ["Label"]

reshapedTrainLabels = train_labels.values.reshape(1, -1)
reshapedTrainLabels = reshapedTrainLabels[0]
reshapedTestLabels = test_labels.values.reshape(1, -1)
reshapedTestLabels = reshapedTestLabels[0]

In [28]:
## Training ##
start = time.process_time()

## Create model with 500 trees, max_depth = 3
model = RandomForestClassifier(n_estimators=500, max_depth=3, random_state=0)

## Fit on training data
model.fit(train, reshapedTrainLabels)

elapsed = (time.process_time() - start)
print("Time used:", elapsed)

Time used: 2.203125


In [33]:
## Validation ##
train_rf_probs = model.predict_proba(train)[:, 1]

rf_probs = model.predict_proba(test)[:, 1]
rf_preds = model.predict(test)

In [34]:
print(len(rf_probs), len(test_labels))

1195 1195


In [40]:
test_labels

,Label
5956,1.0
4332,1.0
2584,1.0
1755,0.0
4687,0.0
1351,0.0
5431,0.0
5721,0.0
79,0.0
626,1.0


array([1., 1., 1., ..., 0., 1., 0.])

In [44]:
df = pd.concat([pd.DataFrame(test_labels.values), pd.DataFrame(rf_preds)], axis=1)
df.columns = ["Labels", "Prediction"]
df

,Labels,Prediction
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,1.0,1.0


In [45]:
count = 0.0
for i in range(len(df)):
    if df.iloc[i, 0] == df.iloc[i,1]:
        count = count + 1

print(count / len(df))

1.0


In [46]:
dump(model, 'randomforest2.joblib')

['randomforest2.joblib']